# Fraud Transaction Pattern Analysis

This notebook explores patterns in credit card transaction data to better understand how transaction characteristics such as  transaction amount and relate them to fraud risk. The analysis uses SQL for aggregation and Python for data manipulation and visualization.


In [ ]:
!pip3 install numpy pandas matplotlib seaborn scikit-learn statsmodels

In [ ]:
import pandas as pd
import os

# make sure data folder exists
os.makedirs("data", exist_ok=True)

# load full dataset
df = pd.read_csv("data/transactions.csv")

# create sample
df_sample = df.sample(50000, random_state=42)

# save sample
df_sample.to_csv("data/transactions_sample.csv", index=False)

os.listdir("data")

In [ ]:
import sqlite3

conn = sqlite3.connect("transactions.db")
df.to_sql("transactions", conn, if_exists="replace", index=False)

## Average Transaction Amount by Fraud Class

This analysis compares the average transaction amount between fraudulent and non-fraudulent transactions to assess whether transaction size is associated with fraud risk.

In [ ]:
query = """
SELECT Class, AVG(Amount) AS avg_amount
FROM transactions
GROUP BY Class;
"""

avg_amounts = pd.read_sql(query, conn)
avg_amounts

In [ ]:
avg_amounts.plot(
    kind="bar",
    x="Class",
    y="avg_amount",
    legend=False,
    title="Average Transaction Amount by Fraud Class"
)

Fraudulent transactions show a higher average amount, suggesting transaction size may be a useful risk indicator.

## Fraud Rate by Transaction Amount

To better understand how transaction size relates to fraud risk, transactions are grouped into amount buckets.  
This analysis calculates the fraud rate within each bucket to identify transaction size thresholds associated with elevated fraud risk.


In [ ]:
query = """
SELECT
    CASE
        WHEN Amount < 50 THEN 'Under $50'
        WHEN Amount < 200 THEN '$50–$200'
        WHEN Amount < 500 THEN '$200–$500'
        ELSE '$500+'
    END AS amount_bucket,
    COUNT(*) AS total_transactions,
    SUM(Class) AS fraud_count,
    ROUND(100.0 * SUM(Class) / COUNT(*), 2) AS fraud_rate_pct
FROM transactions
GROUP BY amount_bucket
ORDER BY fraud_rate_pct DESC;
"""

fraud_by_amount = pd.read_sql(query, conn)
fraud_by_amount


In [ ]:
fraud_by_amount.plot(
    kind="bar",
    x="amount_bucket",
    y="fraud_rate_pct",
    legend=False,
    title="Fraud Rate by Transaction Amount"
)

Fraud rates increase with transaction amount, with the highest fraud rate observed in transactions over $500.

## Conclusion

This analysis examined patterns in credit card transactions to better understand how transaction amount relates to fraud risk. Using SQL and Python, fraudulent and non-fraudulent transactions were compared across multiple transaction size ranges.

The results show that fraudulent transactions tend to have higher average amounts and that fraud rates increase as transaction size grows, with the highest risk observed among large-dollar transactions. While transaction amount alone is not sufficient for identifying fraud, it appears to be a meaningful risk signal when combined with additional features.

This project demonstrates how SQL-based aggregation and exploratory data analysis can be used to uncover actionable insights in fraud detection contexts.
